<a href="https://colab.research.google.com/github/simpleParadox/PSYCO-576-Fall-2020/blob/master/PSYCO_576_ML_for_Psychology_DW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this first Data Workshop, we will look at some ways to inspect data and get feel for how the data is structured.


Goals and learning objectives:


1.   Load EEG data
2.   Inspect EEG data




We will use the 'mne' python package to work with EEG data. \\
https://mne.tools/stable/index.html


Original Paper:



Variables: 



In [2]:
# Install necessary packages
!pip install mne

     |████████████████████████████████| 6.6MB 2.1MB/s 


In [4]:
import mne

In [5]:
mne.__version__

'0.20.8'